# Task
We want to achieve the following workflow:

1. do addition until reaching a certain value

**Note:**

**This might seem simple and general for a python program. But you can imagine, we can swap 'do addition' to something else, like machine learning training.**

**Run through this python notebook and think about the differences between the while loop and dflow recurse steps.**

In [1]:
from dflow import (
    InputParameter,
    Inputs,
    OutputParameter,
    ShellOPTemplate,
    Step,
    Steps,
    Workflow,
)

plus1 is a shell OP using alpine: 3.15 image.
- function: the parameter "script" is to echo the current number of iter `echo 'This is iter {{inputs.parameters.iter}}` , and add one `echo $(({{inputs.parameters.iter}}+1)) ` and then write the number of iter to result.txt

- input: initial number of iter
- output: the number of iter after each iter

In [2]:
plus1 = ShellOPTemplate(
    name="plus1",
    image="alpine:3.15",
    script="echo 'This is iter {{inputs.parameters.iter}}' && "
    "echo $(({{inputs.parameters.iter}}+1)) > /tmp/result.txt",
)
plus1.inputs.parameters = {"iter": InputParameter()}
plus1.outputs.parameters = {"iter": OutputParameter(value_from_path="/tmp/result.txt")}

steps is a kind of Steps called "iter". As shown in the source codes, Steps doesn't require specific parameters. In order to achieve "do addition", we need two parameters, iter and limit with values of 0 and 3, respectively.

In [3]:
steps = Steps(
    name="iter",
    inputs=Inputs(
        parameters={"iter": InputParameter(value=0), "limit": InputParameter(value=3)}
    ),
)

**Note**

1. We are using `dflow.Steps` not `dflow.Step`. 

2. `Steps` is essentially an `OPTemplate` ([See source code](https://github.com/deepmodeling/dflow/blob/master/src/dflow/steps.py)). Using this function, we can construct more complicated workflow. For instance, we are using `dflow.Steps` to create a while loop. 

The addition step:
- it calls add using plus1 template
- its needed parameter is from steps.inputs.parameters["iter"]

In [4]:
addition = Step(
    name="add", template=plus1, parameters={"iter": steps.inputs.parameters["iter"]}
)

the loop step: 
- update the number of iter by addition.outputs.parameters["iter"]
- the loop's condition is when the iter number of additon step less than the limit.

In [5]:
loop = Step(
    name="loop",
    template=steps,
    parameters={"iter": addition.outputs.parameters["iter"]},
    when="%s < %s"
    % (addition.outputs.parameters["iter"], steps.inputs.parameters["limit"]),
)

This step use steps as its template (note that Steps is a subclass of OPTemplate), meanwhile the steps it used contains this step,which gives a recursion. The recursion will stop when the "when" condition is satisfied (after 10 loops in this example)

In [6]:
steps.add(addition)
steps.add(loop)

In [7]:
wf = Workflow("recurse", steps=steps)
wf.submit();

Workflow has been submitted (ID: recurse-m76t5)
